## SIMOC B2 Planning

This file contains the working versions of the data_files to be used in SIMOC-B2, and a basic test of functionality.

Data files include:
* Parameters for agents and currencies (mirroring the current): `agent_desc`, `agent_conn`, `currency_desc`, `agent_events`, and `agent_variation`
* Preset game configurations (new): `mission_1a`, `mission_1b`, `mission_2`

In [13]:
import json

"""
agent_desc
==========
Documentation: https://simoc.space/docs/user_guide/api/data-objects.html#agent-description 

Complete agent list for the first release of SIMOC-B2. 

Some of these we already have agents defined for, 
but whether or not we can reuse/modify the current is TBD.
"""
b2_agent_desc = {
    'inhabitants': {
        'human_agent': {},
        'predators': {},
    },
    'eclss': {
        'b2_co2_removal': {},
        'b2_o2_resupply': {},
    },
    'plants': {        
        'rice': {},
        'wheat': {},
        'sorghum': {},
        'sweet_potato': {},
        'vegetables': {},
        'soybean': {},
        'peanut': {},
        'corn': {},
        'dry_bean': {},
        'orchard': {},
    },
    'structures': {
        'b2_greenhouse': {},
        'b2_crew_quarters': {},
        'b2_biomes': {},
    },
}

# Print the agents we currently have, and those that are missing
current_agents = []
missing_agents = []
with open('data_files/agent_desc.json') as f:
    agent_desc = json.load(f)
    for category, agents in b2_agent_desc.items():
        if category not in agent_desc.keys():
            missing_agents += list(agents.keys())
        for agent, _ in agents.items():
            if agent not in agent_desc[category].keys():
                missing_agents.append(agent)
            else:
                current_agents.append(agent)
print(f'{current_agents=}')
print(f'{missing_agents=}')

current_agents=['human_agent', 'rice', 'wheat', 'sweet_potato', 'soybean', 'peanut', 'dry_bean']
missing_agents=['predators', 'b2_co2_removal', 'b2_o2_resupply', 'sorghum', 'vegetables', 'corn', 'orchard', 'b2_greenhouse', 'b2_crew_quarters', 'b2_biomes']


In [1]:
"""
presets
==========
Documentation: https://simoc.space/docs/user_guide/api/data-objects.html#config

The starting agents and quantities for:
1. Mission 1a: The initial mission without o2 resupply
2. Mission 1b: The initial mission beginning at the time o2 resupply is added
3. Mission 2: The second mission (after a several-month break)
"""

# Everything except 'agents'
default_config = {
    'termination': [{
        'condition': 'time',
        'value': 365,
        'unit': 'day',
    }],
    'priorities': [
        'structures', 'storage', 'inhabitants', 'eclss', 'plants'
    ],
    'seed': 12345,
    'global_entropy': 0,
    'location': 'earth',
    'minutes_per_step': 60,
    'single_agent': 1,
}


# The original mission:
# - Humans: amount, weight, rates of metabolism / respiration
# - Structures: volume & composition of atmosphere
# - Plants: which species (limit to those listed above), amount (area)
# - Pests: any data related to the affect of pests
config_mission_1a = {
    **default_config,
    'agents': {
        'human_agent': {
            'amount': 1,
        }
    },
}

# Figure 3 shows the comparison of yields between mission 1 and 2,
# Mission 1
# Rice 2.4 MT/ha
# Wheat 1.8 MT/ha
# Corn 0 MT/ha
# Sorghum 2.1 MT/ha
# Soybean 2.5 MT/ha

# The original mission from the time o2 is added
# - What day was it added? Duration?
# - Starting atmosphere, currencies, agents should match mission_1a at that time
# - Reconfigured greenhouse, much simplified. Amount, species



# - O2 in from storage (mimic the CO2 resupply valve)
config_mission_1b = {
    **default_config,
    'agents': {
        'human_agent': {
            'amount': 1,
        }
    },
}

# The second mission, things went much better
# - How long after the start of mission_1a?
# - How is garden reconfigured

# Per Figure 1 in Marino et. al, this configuration is 1994
# Rice / Azolla / Tilapia 69+89 m^2 -- Plots 1,10
# Wheat 56+94 m ^2                  -- Plots 2,13
# Sorghum 55 m^2                    -- Plot 3
# Peanut / Corn 45 m ^2             -- Plot 4
# Beans 40+90 m^2                   -- Plots 5,8
# Sweet Potato 52+54 m^2            -- Plots 6,7
# "Vegetables" 60+81 m^2            -- Plots 9,14 
# Soybeans 81 m^2                   -- Plot 11
# Corn 88 m^2                       -- Plot 12
# Rice 78 m^2                       -- Plot 15
# Upland Rice / Corn 83 m^2         -- Plot 16
# Sorghum / Soybean 102 m^2         -- Plot 17
# Peanut / Corn 82 m^2              -- Plot 18

# Table 1 shows that mission 2 eliminates:
#  kale, kohlrabi,  peas,  plantains, winter squash, tumeric, loquat, coconut,
# cacao, grape, pineapple, apple, grape, kumquat, acerola, Bay Tree,

# Mission 2 adds:
# mustard leaf, garlic, okra, sponge gourd, pumpkin, bottle gourd, zucchini, oregano,
# Coriander, Parsley, Rosemary, Fennel, Ginger, and Tapioca, rice beans, kidney beans,
# navy beans, lima beans, 

# Table 1 also shows a variety of plants that were maintained in both missions without
# elaborating on quantities due to being in the orchard or balconies

# Figure 3 shows the comparison of yields between mission 1 and 2,
# Mission 2
# Rice 7.4 MT/ha
# Wheat 3.5 MT/ha
# Corn 4 MT/ha
# Sorghum 4.5 MT/ha
# Soybean 2 MT/ha
 

# - Predators: any information about which predators and their impact
# - How much additional time spent weeding?
config_mission_2 = {
    **default_config,
    'agents': {
        'human_agent': {
            'amount': 1,
        }
    },
}